In [ ]:
%matplotlib inline

import pymysql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale
from IPython.display import display, HTML

In [4]:
conn = pymysql.connect(host='104.196.5.55', user='root', passwd='', db='nfl')

cur = conn.cursor()

In [30]:
#Create Offense dataframe with only WRs and relevant variables, create redzone dataframe
strSQL = ("Select * from pass p where yds > 39")

df_pass = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

df_pass

,pid,psr,trg,loc,yds,comp,succ,spk,dfb
0,75,CC-1400,TM-0900,NL,44,1,1,0,\r
1,98,CC-1400,SJ-0400,NL,48,1,1,0,\r
2,622,BF-0100,AF-1300,NL,48,1,1,0,\r
3,630,VT-0200,DW-0800,NL,61,1,1,0,\r
4,1051,CM-2200,MR-1400,NL,48,1,1,0,\r
5,1085,DC-4400,RM-5000,NL,66,1,1,0,\r
6,1819,TB-0200,QI-0100,NL,53,1,1,0,\r
7,2309,BG-1900,RS-2100,NL,47,1,1,0,\r
8,2365,KW-0500,MF-0300,NL,72,1,1,0,\r
9,2379,KW-0500,AH-0200,NL,80,1,1,0,\r


In [27]:
#Create Offense dataframe with only WRs and relevant variables, create redzone dataframe
strSQL = ("Select * from offense o WHERE o.YEAR = 2015")

df_all_offense_2015 = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

df_all_offense_2015

,uid,gid,player,pa,pc,py,ints,tdp,ra,sra,...,fp,fp2,fp3,game,seas,year,team,posd,jnum,dcp
0,82968,3990,DW-3600,0,0,0,0,0,21,14,...,13.20,13.70,17.20,1,10,2015,PIT,RB,34,1
1,82969,3990,BR-1100,38,26,351,1,1,0,0,...,18.04,19.04,21.04,1,12,2015,PIT,QB,7,1
2,82970,3990,AB-3500,0,0,0,0,0,0,0,...,19.30,23.80,31.30,1,6,2015,PIT,LWR,84,1
3,82971,3990,HM-0200,0,0,0,0,0,0,0,...,8.40,12.40,16.40,1,11,2015,PIT,TE,83,1
4,82972,3990,WJ-0300,0,0,0,0,0,4,1,...,6.70,6.70,6.70,1,4,2015,PIT,FB,46,1
5,82973,3990,TB-2300,32,25,288,0,4,3,2,...,27.62,27.62,27.62,1,16,2015,NE,QB,12,1
6,82974,3990,JE-0100,0,0,0,0,0,1,1,...,10.60,16.10,21.60,1,7,2015,NE,RWR,11,1
7,82975,3990,DL-1700,0,0,0,0,0,15,8,...,12.00,14.00,16.00,1,5,2015,NE,RB,33,2
8,82976,3990,RG-2200,0,0,0,0,0,0,0,...,27.40,29.90,32.40,1,6,2015,NE,TE,87,1
9,82977,3990,DA-0900,0,0,0,0,0,0,0,...,2.40,3.40,4.40,1,8,2015,NE,LWR,80,1


In [26]:
#Create Offense dataframe with only WRs and relevant variables, create redzone dataframe
strSQL = ("Select * from pass p")

df_pass = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

df_pass

,pid,psr,trg,loc,yds,comp,succ,spk,dfb
0,3,JG-0600,JR-2000,NL,10,1,1,0,\r
1,5,JG-0600,TO-0200,NL,19,1,1,0,\r
2,7,JG-0600,CG-0400,NL,24,1,1,0,\r
3,9,JG-0600,FB-0200,NL,4,1,1,0,\r
4,15,CC-1400,BC-1100,NL,0,0,0,0,\r
5,16,CC-1400,TM-0900,NL,0,0,0,0,\r
6,20,JG-0600,JR-2000,NL,0,0,0,0,\r
7,21,JG-0600,GC-0500,NL,8,1,0,0,\r
8,24,CC-1400,SJ-0400,NL,0,0,0,0,\r
9,25,CC-1400,TM-0900,NL,17,1,1,0,\r


In [24]:
def qb_points():
    #Passing points
    pass_yrds = df_all_offense_2015['py']/5*.2
    pass_40_tds = 
    pass_50_tds = 
    pass_tds = df_all_offense_2015['tdp']*4
    pass_ints = df_all_offense_2015['ints']*-2
    pass_two_pt =
    
    #Rushing points
    rush_yrds = 
    rush_tds = 
    rush_40_tds = 
    rush_50_tds = 
    rush_two_pt =
    
    #Receiving yards =
    
    
    

,uid,gid,player,pa,pc,py,ints,tdp,ra,sra,...,fp,fp2,fp3,game,seas,year,team,posd,jnum,dcp
0,82968,3990,DW-3600,0,0,0,0,0,21,14,...,13.20,13.70,17.20,1,10,2015,PIT,RB,34,1
1,82969,3990,BR-1100,38,26,351,1,1,0,0,...,18.04,19.04,21.04,1,12,2015,PIT,QB,7,1
2,82970,3990,AB-3500,0,0,0,0,0,0,0,...,19.30,23.80,31.30,1,6,2015,PIT,LWR,84,1
3,82971,3990,HM-0200,0,0,0,0,0,0,0,...,8.40,12.40,16.40,1,11,2015,PIT,TE,83,1
4,82972,3990,WJ-0300,0,0,0,0,0,4,1,...,6.70,6.70,6.70,1,4,2015,PIT,FB,46,1
5,82973,3990,TB-2300,32,25,288,0,4,3,2,...,27.62,27.62,27.62,1,16,2015,NE,QB,12,1
6,82974,3990,JE-0100,0,0,0,0,0,1,1,...,10.60,16.10,21.60,1,7,2015,NE,RWR,11,1
7,82975,3990,DL-1700,0,0,0,0,0,15,8,...,12.00,14.00,16.00,1,5,2015,NE,RB,33,2
8,82976,3990,RG-2200,0,0,0,0,0,0,0,...,27.40,29.90,32.40,1,6,2015,NE,TE,87,1
9,82977,3990,DA-0900,0,0,0,0,0,0,0,...,2.40,3.40,4.40,1,8,2015,NE,LWR,80,1
